In [1]:
import pandas as pd
import sklearn
from sklearn import tree
import seaborn as sns
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import keras
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, f1_score

%matplotlib inline

In [3]:
# Загрузили датасет
df = pd.read_csv('dataset.csv')

# Удалили лишний индексный столбец
df = df.drop(columns=['n'])

# Объединил травы
df.loc[ df['Culture']=='травы многолетние', 'Culture'] = 'многолетние травы'

# заменил скачки и наны средними по культуре
mask = (df[['121', '128', '135', '142']] > 0.5)
df[mask] = np.nan
df_filled = df.groupby('Culture').transform(lambda x: x.fillna(x.mean()))
df_filled['Culture'] = df['Culture']

# если всё хорошо, меняем основной датасет
df = df_filled

In [4]:
X = df.drop(['Culture'], axis=1)
# axis=1 означает что мы передаём не индексы, а колонки

onehotencoder = OneHotEncoder()
y = pd.DataFrame(onehotencoder.fit_transform(df[['Culture']]).toarray(), columns=onehotencoder.get_feature_names_out(['Culture']))

In [5]:
# разбили X и y на тренировочные и тестовые части
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 25)

In [8]:
csvn = 500

In [17]:
model = keras.Sequential(
    [
        keras.layers.Dense(128, activation='LeakyReLU'),
        keras.layers.Normalization(),
        keras.layers.Dense(512, activation='LeakyReLU'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(256, activation='LeakyReLU'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(32, activation='LeakyReLU'),
        keras.layers.Dense(12),
        keras.layers.Dense(12, activation='softmax')
    ]

)
model.compile(keras.optimizers.Adagrad(learning_rate=0.015), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [22]:
model.fit(x=X_train, y=y_train, epochs=50)
'''X_test_private = pd.read_csv('private_dataset.csv')
X_test_private.drop(X_test_private.columns[0], axis=1, inplace=True)
y_pred = pd.DataFrame(model.predict(X_test_private))
y_pred = onehotencoder.inverse_transform(y_pred)
#pd.DataFrame(onehotencoder.inverse_transform(y_pred)).to_csv(('answer'+str(csvn)+'.csv'),index=False) '''
csvn += 1

Epoch 1/50
355/355 [==============================] - 3s 7ms/step - loss: 0.4168 - categorical_accuracy: 0.8592
Epoch 2/50
355/355 [==============================] - 3s 8ms/step - loss: 0.4126 - categorical_accuracy: 0.8605
Epoch 3/50
355/355 [==============================] - 3s 9ms/step - loss: 0.4143 - categorical_accuracy: 0.8619
Epoch 4/50
355/355 [==============================] - 3s 9ms/step - loss: 0.4127 - categorical_accuracy: 0.8604
Epoch 5/50
355/355 [==============================] - 3s 7ms/step - loss: 0.4088 - categorical_accuracy: 0.8647
Epoch 6/50
355/355 [==============================] - 3s 7ms/step - loss: 0.4103 - categorical_accuracy: 0.8653
Epoch 7/50
355/355 [==============================] - 3s 8ms/step - loss: 0.4071 - categorical_accuracy: 0.8641
Epoch 8/50
355/355 [==============================] - 4s 10ms/step - loss: 0.4085 - categorical_accuracy: 0.8602
Epoch 9/50
355/355 [==============================] - 3s 7ms/step - loss: 0.4042 - categorical_accuracy

In [16]:
import pickle

filename = 'DDTSmodel2.pkl'
pickle.dump(model, open(filename, 'wb'))

In [30]:
X_test_private = pd.read_csv('private_dataset.csv')
X_test_private.drop(X_test_private.columns[0], axis=1, inplace=True)
y_pred = pd.DataFrame(model.predict(X_test_private))
#y_pred = onehotencoder.inverse_transform(y_pred)
pd.DataFrame(onehotencoder.inverse_transform(y_pred)).to_csv('test_answer1.csv',index=False) # ОТВЕТ НЕЙРОСЕТИ
culture_df = pd.read_csv('test_answer1.csv')

def update_culture_based_on_sum(main_df, culture_df):
    updated_culture_df = culture_df.copy()  # Создаем копию второго датафрейма

    for index, row in main_df.iterrows():
        # Шаг 1: Считаем сумму значений в первых пяти столбцах
        sum_of_first_5 = row.iloc[1:6].sum()

        if sum_of_first_5 > 1.6:
            updated_culture_df.iloc[index, 0] = "многолетние травы"
            print(sum_of_first_5)


    return updated_culture_df

private_df = pd.read_csv('private_dataset.csv')
df_updated_cultures = update_culture_based_on_sum(private_df, culture_df)
pd.DataFrame(df_updated_cultures).to_csv('finaltest1.csv',index=False) # ИТОГОВЫЙ ОТВЕТ ПОСЛЕ УЛУЧШЕНИЯ

#pd.DataFrame(onehotencoder.inverse_transform(df_updated_cultures)).to_csv(('answer'+str(csvn)+'.csv'),index=False)


12/12 [==============================] - 0s 3ms/step
1.6633844462793599
1.8801895351686682
1.8048020795949213
1.767096503594181
1.8470886644372764
1.883945930249141
1.6259684106948524
1.643139505802201
1.7725920105955788
1.9278480457828078
1.9807647572368756
1.980089539349657
2.104422711445713
1.9377946110369955
2.123197982256433
1.6828768343651157
1.7288360269961478
1.8077023977233049
